In [1]:
# 수정 전: from langchain.schemas import Document (X)
# 수정 후: 아래 둘 중 하나를 사용하세요. (v0.3에서는 1번 권장)

# 방법 1: 가장 권장되는 최신 경로
from langchain_core.documents import Document

# 방법 2: 구버전 호환 경로 (단수형 schema)
# from langchain.schema import Document

sample_docs = [
    Document(
        page_content="국내선은 액체류(물, 음료, 화장품 등)의 객실 반입 제한이 없습니다.", 
        metadata={"scope": "domestic", "doc_type": "liquid", "jurisdiction": "KR"}
    ),
    Document(
        page_content="국제선은 100ml 이하 용기에 담긴 액체만 1L 투명 지퍼백 1개에 넣어 반입 가능합니다.", 
        metadata={"scope": "international", "doc_type": "liquid", "jurisdiction": "KR"}
    ),
    Document(
        page_content="리튬 보조배터리는 화재 위험으로 반드시 기내 휴대해야 하며, 위탁수하물(부치는 짐)은 금지됩니다.", 
        metadata={"scope": "both", "doc_type": "battery", "jurisdiction": "ALL"}
    )
]

print(f"성공적으로 {len(sample_docs)}개의 문서를 생성했습니다!")

성공적으로 3개의 문서를 생성했습니다!


In [2]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers.ensemble import EnsembleRetriever

def get_hybrid_retriever(docs, scope_choice):
    """
    사용자의 노선(scope)에 따라 필터링된 하이브리드 검색기를 반환합니다.
    """
    # 1. 메타데이터 필터링: 선택한 노선과 공통(both) 규정만 추출
    filtered_docs = [d for d in docs if d.metadata["scope"] in [scope_choice, "both"]]
    
    # 2. 키워드 검색기 (BM25) 설정
    bm25_retriever = BM25Retriever.from_documents(filtered_docs)
    bm25_retriever.k = 2  # 상위 2개 결과 추출
    
    # 3. 벡터 검색기 (FAISS) 설정
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(filtered_docs, embeddings)
    faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
    
    # 4. 앙상블 검색기 (Ensemble) 생성
    # 가중치를 5:5로 섞어 키워드와 의미 검색의 장점을 모두 취함
    ensemble_retriever = EnsembleRetriever(
        retrievers=[bm25_retriever, faiss_retriever],
        weights=[0.5, 0.5]
    )
    
    return ensemble_retriever

print("✅ 검색기 생성 함수 정의 완료!")

✅ 검색기 생성 함수 정의 완료!


In [6]:
"""
항공 보안 검색 RAG 파이프라인 - 통합 테스트 코드
================================================
BM25 + FAISS 하이브리드 검색 → 의도 분석(라우터) → LLM 답변 생성
"""

import os
from pydantic import BaseModel, Field
from typing import Optional
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain.retrievers.ensemble import EnsembleRetriever

# ==========================================
# 0. API 키 설정
# ==========================================
load_dotenv()  # .env 파일에서 OPENAI_API_KEY 로드

# ==========================================
# 1. LLM 및 임베딩 모델 초기화
# ==========================================
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# ==========================================
# 2. 의도 분석 라우터 (Pydantic Structured Output)
# ==========================================
class FlightIntent(BaseModel):
    is_dangerous: bool = Field(
        default=False,
        description="폭발물, 인화성 물질(가스, 라이터 등), 무기류 등 명백한 위험물 여부"
    )
    flight_type: Optional[str] = Field(
        default=None,
        description="비행 종류. '국내선' 또는 '국제선'. 모르면 None"
    )
    baggage_type: Optional[str] = Field(
        default=None,
        description="수하물 유형. '기내' 또는 '위탁'. 모르면 None"
    )

structured_llm = llm.with_structured_output(FlightIntent)

intent_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 항공 보안 검색 안내 챗봇의 라우터입니다. 
    사용자의 질문을 분석하여 위험물 여부, 국내/국제선 여부, 기내/위탁 여부를 추출하세요. 
    정보가 명확하지 않으면 억지로 채우지 말고 반드시 None으로 두세요."""),
    ("user", "{query}")
])
intent_chain = intent_prompt | structured_llm

# ==========================================
# 3. 지식 베이스 (국제선 규정 문서)
# ==========================================
docs_intl = [
    Document(
        page_content="모든 액체, 분무, 겔류(LAGs)는 용량이 100ml 이하인 용기에 담아야 기내 반입이 가능합니다. 100ml를 초과하는 빈 용기는 반입할 수 있으나, 내용물이 조금이라도 남은 100ml 초과 용기는 반입할 수 없습니다.",
        metadata={"category": "국제선_기내", "source": "제5조 1항"}
    ),
    Document(
        page_content="반입 금지 액체류 예시: 물, 음료수, 국 종류(스프류), 김치류, 된장, 고추장 등 각종 장류, 화장품(로션, 샴푸, 마스카라 등). 이들은 100ml 초과 시 반드시 위탁 수하물로 부쳐야 합니다.",
        metadata={"category": "국제선_일반", "source": "별표 1의1"}
    ),
    Document(
        page_content="의사 처방전이 있는 의약품, 시판 액상 감기약, 콘택트렌즈 보존액, 어린아이 용품(우유, 모유, 이유식 등)은 비행 여정에 필요한 용량만큼 100ml를 초과해도 기내 반입이 예외적으로 허용됩니다.",
        metadata={"category": "국제선_예외", "source": "별표 1의2"}
    ),
]

# ==========================================
# 4. 하이브리드 리트리버 구축 (BM25 + FAISS)
# ==========================================
print("📦 벡터스토어 및 검색기 초기화 중...")

vectorstore_intl = FAISS.from_documents(docs_intl, embeddings)
faiss_retriever_intl = vectorstore_intl.as_retriever(search_kwargs={"k": 2})

bm25_retriever_intl = BM25Retriever.from_documents(docs_intl)
bm25_retriever_intl.k = 2

ensemble_retriever_intl = EnsembleRetriever(
    retrievers=[bm25_retriever_intl, faiss_retriever_intl],
    weights=[0.5, 0.5]
)
print("✅ 하이브리드 리트리버 준비 완료!\n")

# ==========================================
# 5. 답변 생성 체인
# ==========================================
def format_docs(docs):
    return "\n\n".join(
        f"[출처: {doc.metadata.get('source')}]\n{doc.page_content}" for doc in docs
    )

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 정확하고 친절한 공항 보안 검색 전문가입니다.
    반드시 제공된 [관련 규정]을 바탕으로 답변하세요. 규정을 인용할 때는 출처를 함께 언급해 주세요.
    
    [관련 규정]
    {context}"""),
    ("user", "{question}")
])
rag_chain = qa_prompt | llm | StrOutputParser()

# ==========================================
# 6. 메인 처리 함수
# ==========================================
def process_chat(user_query: str) -> str:
    print(f"\n🗣️ 사용자 질문: {user_query}")
    print("-" * 50)

    # Step 1: 의도 분석
    intent = intent_chain.invoke({"query": user_query})
    print(f"🔍 [라우터 분석] 위험물:{intent.is_dangerous} | 비행:{intent.flight_type} | 수하물:{intent.baggage_type}")

    # Step 2: 위험물 즉시 차단
    if intent.is_dangerous:
        return "🚨 [위험물 안내] 해당 물품은 폭발/인화성 물질로 분류되어 기내 및 위탁 반입이 모두 엄격히 금지됩니다."

    # Step 3: 필수 정보 누락 시 되묻기
    missing_info = []
    if not intent.flight_type:
        missing_info.append("국내선/국제선 여부")
    if not intent.baggage_type:
        missing_info.append("기내 반입/위탁 여부")

    if missing_info:
        return f"💬 정확한 규정 확인을 위해 {'와(과) '.join(missing_info)}를 먼저 알려주시겠어요?"

    # Step 4: 타겟 DB 검색
    retrieved_docs = []
    if intent.flight_type == "국제선":
        print("📥 [검색 진행] 국제선 하이브리드 리트리버 가동 중...")
        retrieved_docs = ensemble_retriever_intl.invoke(user_query)
    elif intent.flight_type == "국내선":
        return "💬 국내선은 액체류 반입 제한이 없습니다. 자유롭게 기내에 들고 타셔도 됩니다."

    # Step 5: 최종 답변 생성
    print("🤖 [답변 생성] LLM이 문서를 읽고 답변을 작성 중입니다...")
    formatted_context = format_docs(retrieved_docs)

    final_answer = rag_chain.invoke({
        "context": formatted_context,
        "question": user_query
    })

    return final_answer


# ==========================================
# 7. 🚀 테스트 실행
# ==========================================
if __name__ == "__main__":
    print("=" * 60)
    print("  ✈️  항공 보안 RAG 챗봇 테스트")
    print("=" * 60)

    # 테스트 1: 정보가 부족한 질문 → 되묻기 로직 확인
    print("\n📋 [테스트 1] 정보 부족 → 되묻기")
    ans1 = process_chat("고추장 가져가도 돼?")
    print(f"👉 챗봇: {ans1}\n")

    # 테스트 2: 위험물 질문 → 즉시 차단 확인
    print("\n📋 [테스트 2] 위험물 → 차단")
    ans2 = process_chat("부탄가스를 가방에 넣어도 돼?")
    print(f"👉 챗봇: {ans2}\n")

    # 테스트 3: 완벽한 질문 → RAG 전체 파이프라인 동작
    print("\n📋 [테스트 3] 완전한 질문 → RAG 파이프라인")
    ans3 = process_chat("이번에 일본 가는데, 아기가 먹을 이유식을 기내에 들고 타도 될까?")
    print(f"👉 챗봇: {ans3}\n")

    print("=" * 60)
    print("  ✅ 모든 테스트 완료!")
    print("=" * 60)


📦 벡터스토어 및 검색기 초기화 중...
✅ 하이브리드 리트리버 준비 완료!

  ✈️  항공 보안 RAG 챗봇 테스트

📋 [테스트 1] 정보 부족 → 되묻기

🗣️ 사용자 질문: 고추장 가져가도 돼?
--------------------------------------------------
🔍 [라우터 분석] 위험물:False | 비행:None | 수하물:기내
👉 챗봇: 💬 정확한 규정 확인을 위해 국내선/국제선 여부를 먼저 알려주시겠어요?


📋 [테스트 2] 위험물 → 차단

🗣️ 사용자 질문: 부탄가스를 가방에 넣어도 돼?
--------------------------------------------------
🔍 [라우터 분석] 위험물:True | 비행:None | 수하물:None
👉 챗봇: 🚨 [위험물 안내] 해당 물품은 폭발/인화성 물질로 분류되어 기내 및 위탁 반입이 모두 엄격히 금지됩니다.


📋 [테스트 3] 완전한 질문 → RAG 파이프라인

🗣️ 사용자 질문: 이번에 일본 가는데, 아기가 먹을 이유식을 기내에 들고 타도 될까?
--------------------------------------------------
🔍 [라우터 분석] 위험물:False | 비행:국제선 | 수하물:기내
📥 [검색 진행] 국제선 하이브리드 리트리버 가동 중...
🤖 [답변 생성] LLM이 문서를 읽고 답변을 작성 중입니다...
👉 챗봇: 네, 아기가 먹을 이유식은 기내에 들고 타실 수 있습니다. 의사 처방전이 있는 의약품, 시판 액상 감기약, 콘택트렌즈 보존액, 어린아이 용품(우유, 모유, 이유식 등)은 비행 여정에 필요한 용량만큼 100ml를 초과해도 기내 반입이 예외적으로 허용됩니다. 따라서 아기의 이유식을 준비하셔도 괜찮습니다. 안전한 여행 되시길 바랍니다! [출처: 별표 1의2]

  ✅ 모든 테스트 완료!


In [7]:
"""
test_rag.py - 멀티-레이어 RAG 파이프라인 통합 테스트
====================================================
sample_data → router → retriever → chain 전체 흐름 테스트
"""

import os
import sys

from dotenv import load_dotenv

# .env에서 API 키 로드
load_dotenv()

# ==========================================
# 모듈 임포트
# ==========================================
from sample_data import get_all_documents
from router import build_router, resolve_filters, apply_airline_filter
from retriever import filter_documents, build_retriever
from chain import format_docs_with_source, build_qa_chain, build_danger_chain


# ==========================================
# 전역 초기화
# ==========================================
print("=" * 60)
print("  ✈️  항공 보안 RAG 챗봇 - 멀티레이어 파이프라인 테스트")
print("=" * 60)

ALL_DOCS = get_all_documents()
print(f"\n📦 전체 문서 로드 완료: {len(ALL_DOCS)}건")

intent_chain = build_router()
qa_chain = build_qa_chain()
danger_chain = build_danger_chain()

print("✅ 라우터 및 체인 초기화 완료!\n")


# ==========================================
# 메인 처리 함수
# ==========================================
def process_chat(user_query: str) -> str:
    """사용자 질문을 받아 전체 파이프라인을 실행합니다."""
    print(f"\n🗣️  사용자 질문: {user_query}")
    print("-" * 55)

    # ── Step 1: 의도 분석 ──
    intent = intent_chain.invoke({"query": user_query})
    print(
        f"🔍 [라우터] 위험물:{intent.is_dangerous} | "
        f"노선:{intent.flight_type} | 수하물:{intent.baggage_type} | "
        f"카테고리:{intent.category} | 항공사:{intent.airline}"
    )

    # ── Step 2: 위험물 즉시 차단 ──
    if intent.is_dangerous:
        # 위험물 관련 문서 검색 후 경고 답변 생성
        danger_filters = {"category": "general", "flight_type": ["common"]}
        retriever = build_retriever(ALL_DOCS, danger_filters, k=2)
        docs = retriever.invoke(user_query)
        context = format_docs_with_source(docs)
        answer = danger_chain.invoke({"context": context, "question": user_query})
        return f"🚨 {answer}"

    # ── Step 3: 필수 정보 부족 시 되묻기 ──
    missing_info = []
    if not intent.flight_type:
        missing_info.append("국내선/국제선 여부")
    if not intent.baggage_type:
        missing_info.append("기내 반입/위탁 여부")

    if missing_info:
        return (
            f"💬 정확한 규정 확인을 위해 "
            f"{'와(과) '.join(missing_info)}를 먼저 알려주시겠어요?"
        )

    # ── Step 4: 메타데이터 필터 구성 ──
    filters = resolve_filters(intent)
    print(f"🏷️  [필터] {filters}")

    # 항공사 특수 필터 처리
    working_docs = apply_airline_filter(ALL_DOCS, filters)

    # ── Step 5: 하이브리드 검색 ──
    print("📥 [검색] 하이브리드 리트리버 가동 중...")
    retriever = build_retriever(working_docs, filters, k=3)
    retrieved_docs = retriever.invoke(user_query)

    print(f"📄 [검색 결과] {len(retrieved_docs)}건 검색됨")
    for i, doc in enumerate(retrieved_docs, 1):
        src = doc.metadata.get("source", "?")
        cat = doc.metadata.get("category", "?")
        ft = doc.metadata.get("flight_type", "?")
        print(f"   #{i} source={src} | category={cat} | flight_type={ft}")

    # ── Step 6: LLM 답변 생성 ──
    print("🤖 [답변 생성] LLM이 문서를 읽고 답변을 작성 중...")
    context = format_docs_with_source(retrieved_docs)
    answer = qa_chain.invoke({"context": context, "question": user_query})

    return answer


# ==========================================
# 테스트 실행
# ==========================================
def run_tests():
    """6개 시나리오를 순차 실행합니다."""
    tests = [
        {
            "name": "정보 부족 → 되묻기",
            "query": "고추장 가져가도 돼?",
        },
        {
            "name": "위험물 → 즉시 차단",
            "query": "부탄가스를 가방에 넣어도 돼?",
        },
        {
            "name": "국제선 + 액체류 필터링",
            "query": "국제선 기내에 화장품 가져가도 될까요?",
        },
        {
            "name": "항공사(KE) + 배터리 필터링",
            "query": "대한항공 타는데 보조배터리 위탁 수하물로 부칠 수 있나요?",
        },
        {
            "name": "국내선 필터링",
            "query": "국내선 타는데 음료수 기내에 들고 탈 수 있어?",
        },
        {
            "name": "보안검색 후 면세점 음료",
            "query": "국제선인데 보안검색대 지나서 면세점에서 산 음료수 기내에 가져갈 수 있나요?",
        },
    ]

    for i, test in enumerate(tests, 1):
        print(f"\n{'=' * 60}")
        print(f"  📋 테스트 {i}/{len(tests)}: {test['name']}")
        print(f"{'=' * 60}")

        answer = process_chat(test["query"])
        print(f"\n👉 챗봇 답변:\n{answer}")

    print(f"\n{'=' * 60}")
    print("  ✅ 모든 테스트 완료!")
    print(f"{'=' * 60}")


if __name__ == "__main__":
    run_tests()


  ✈️  항공 보안 RAG 챗봇 - 멀티레이어 파이프라인 테스트

📦 전체 문서 로드 완료: 16건
✅ 라우터 및 체인 초기화 완료!


  📋 테스트 1/6: 정보 부족 → 되묻기

🗣️  사용자 질문: 고추장 가져가도 돼?
-------------------------------------------------------
🔍 [라우터] 위험물:False | 노선:None | 수하물:None | 카테고리:food | 항공사:None

👉 챗봇 답변:
💬 정확한 규정 확인을 위해 국내선/국제선 여부와(과) 기내 반입/위탁 여부를 먼저 알려주시겠어요?

  📋 테스트 2/6: 위험물 → 즉시 차단

🗣️  사용자 질문: 부탄가스를 가방에 넣어도 돼?
-------------------------------------------------------
🔍 [라우터] 위험물:True | 노선:None | 수하물:위탁 | 카테고리:general | 항공사:None
📂 검색 대상 문서: 3건 (전체 16건 중)

👉 챗봇 답변:
🚨 부탄가스는 항공 보안 규정에 따라 기내 및 위탁수하물 모두 반입이 금지된 위험물입니다. 따라서 가방에 넣어도 절대 반입할 수 없습니다. 

관련 규정에 따르면, 국토부 고시 [p.8]에서 인화성 물질인 부탄가스는 기내 반입 및 위탁수하물 모두 금지된다고 명시되어 있습니다. 안전한 비행을 위해 이러한 규정을 반드시 준수해 주시기 바랍니다.

  📋 테스트 3/6: 국제선 + 액체류 필터링

🗣️  사용자 질문: 국제선 기내에 화장품 가져가도 될까요?
-------------------------------------------------------
🔍 [라우터] 위험물:False | 노선:국제선 | 수하물:기내 | 카테고리:liquid | 항공사:None
🏷️  [필터] {'flight_type': ['international', 'common'], 'category': 'liquid'}
📥 [검색] 하이브리드 리트리버 가동 중...
📂 검색

In [9]:
import os
from typing import Optional, List
from pydantic import BaseModel, Field
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.chat_message_histories import ChatMessageHistory

# 1. 초기화
load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 2. 의도 분석 스키마 (메모리를 통해 채워질 구조)
class FlightIntent(BaseModel):
    is_dangerous: bool = Field(default=False)
    item_name: Optional[str] = Field(default=None, description="물품 명칭 (예: 고추장, 술, 향수)")
    flight_type: Optional[str] = Field(default=None, description="'국내선' 또는 '국제선'")
    baggage_type: Optional[str] = Field(default=None, description="'기내' 또는 '위탁'")

# 3. 지식 베이스 (면세점 규정 추가)
def get_updated_knowledge_base():
    return [
        Document(
            page_content="국제선 기내 액체류 제한: 100ml 이하 용기만 가능, 1L 지퍼백 1개에 담아야 함.",
            metadata={"scope": "국제선", "baggage_type": "기내", "source": "보안규정 제5조"}
        ),
        Document(
            page_content="면세점 액체류 특례(STEB): 보안검색대 통과 후 면세점에서 구입한 주류, 향수 등 액체류는 '액체물 보안봉투(STEB)'에 영수증과 함께 봉인된 경우 100ml를 초과해도 기내 반입이 가능합니다. 최종 목적지까지 봉인을 뜯어서는 안 됩니다.",
            metadata={"scope": "국제선", "baggage_type": "기내", "source": "면세품 반입 규정"}
        ),
        Document(
            page_content="국내선 액체류: 국내선은 액체류 기내 반입에 용량 제한이 없습니다.",
            metadata={"scope": "국내선", "baggage_type": "기내", "source": "국내선 가이드"}
        )
    ]

# 4. 멀티턴 RAG 클래스
class MultiTurnAviationBot:
    def __init__(self):
        self.all_docs = get_updated_knowledge_base()
        self.history = ChatMessageHistory() # 세션별 관리가 필요하면 dict로 확장 가능
        
        # 의도 분석용 프롬프트 (대화 기록 참조)
        self.router_prompt = ChatPromptTemplate.from_messages([
            ("system", "이전 대화와 현재 질문을 참고하여 사용자의 의도를 분석하세요. 정보가 없으면 None으로 둡니다."),
            MessagesPlaceholder(variable_name="history"),
            ("user", "{query}")
        ])
        self.router = self.router_prompt | llm.with_structured_output(FlightIntent)

        self.qa_chain = ChatPromptTemplate.from_messages([
            ("system", "공항 보안 전문가입니다. [규정]에 근거해 답하세요.\n\n[규정]\n{context}"),
            MessagesPlaceholder(variable_name="history"),
            ("user", "{question}")
        ]) | llm | StrOutputParser()

    def run(self, user_input: str):
        # 1. 의도 분석 (이전 대화 기록 포함)
        intent = self.router.invoke({"query": user_input, "history": self.history.messages})
        
        # 2. 위험물 선제 차단
        if intent.is_dangerous:
            return "🚨 해당 물품은 반입 금지 위험물입니다."

        # 3. 필수 정보(노선/수하물) 부족 시 질문 및 메모리 저장
        if not intent.flight_type or not intent.baggage_type or not intent.item_name:
            self.history.add_user_message(user_input)
            
            missing = []
            if not intent.item_name: missing.append("어떤 물건")
            if not intent.flight_type: missing.append("국내선/국제선 여부")
            if not intent.baggage_type: missing.append("기내/위탁 여부")
            
            response = f"💬 정확한 확인을 위해 {'와(과) '.join(missing)}을(를) 알려주시겠어요?"
            self.history.add_ai_message(response)
            return response

        # 4. 필터링된 검색 (면세점 키워드가 있으면 STEB 규정 우선 검색)
        filtered_docs = [d for d in self.all_docs if d.metadata["scope"] in [intent.flight_type, "common"]]
        
        # 하이브리드 리트리버 가동
        bm25 = BM25Retriever.from_documents(filtered_docs)
        vectorstore = FAISS.from_documents(filtered_docs, embeddings)
        ensemble = EnsembleRetriever(retrievers=[bm25, vectorstore.as_retriever()], weights=[0.5, 0.5])
        
        context_docs = ensemble.invoke(user_input)
        context_text = "\n".join([f"[{d.metadata['source']}] {d.page_content}" for d in context_docs])

        # 5. 최종 답변 생성 및 메모리 업데이트
        final_answer = self.qa_chain.invoke({
            "context": context_text,
            "question": user_input,
            "history": self.history.messages
        })
        
        self.history.add_user_message(user_input)
        self.history.add_ai_message(final_answer)
        return final_answer

In [14]:
def interactive_test():
    # 1. 봇 초기화
    bot = MultiTurnAviationBot()
    
    print("="*60)
    print("✈️  항공 보안 검색 RAG 챗봇 테스트 모드")
    print("대화를 종료하려면 '종료', 'exit', 'q'를 입력하세요.")
    print("="*60)

    while True:
        # 2. 사용자 입력 받기
        user_input = input("\n👤 사용자: ").strip()
        
        # 종료 조건
        if user_input.lower() in ['종료', 'exit', 'q', 'quit']:
            print("\n👋 테스트를 종료합니다. 즐거운 여행 되세요!")
            break
            
        if not user_input:
            continue

        try:
            # 3. 챗봇 엔진 실행
            response = bot.run(user_input)
            
            # 4. 결과 출력
            print(f"\n🤖 챗봇: {response}")
            
        except Exception as e:
            print(f"\n❌ 에러 발생: {e}")
            print("API 키 설정이나 라이브러리 설치 상태를 확인해주세요.")

# 실행
if __name__ == "__main__":
    interactive_test()

✈️  항공 보안 검색 RAG 챗봇 테스트 모드
대화를 종료하려면 '종료', 'exit', 'q'를 입력하세요.

🤖 챗봇: 💬 정확한 확인을 위해 어떤 물건와(과) 국내선/국제선 여부와(과) 기내/위탁 여부을(를) 알려주시겠어요?

🤖 챗봇: 💬 정확한 확인을 위해 어떤 물건와(과) 기내/위탁 여부을(를) 알려주시겠어요?

🤖 챗봇: 라이터는 국제선 비행기 기내에 반입할 수 없습니다. 보안 규정에 따라 라이터는 위탁 수하물로도 반입할 수 없으며, 기내에 반입할 수 있는 물품이 아닙니다. 따라서 라이터는 여행 전에 반드시 처분하시거나 다른 방법으로 안전하게 보관하시기 바랍니다.

👋 테스트를 종료합니다. 즐거운 여행 되세요!
